In [1]:
from pymilvus import model
from pymilvus import MilvusClient, DataType

### Create Milvus connection to a file

In [2]:
client = MilvusClient("/tmp/milvus_demo.db")

client.drop_collection(collection_name="my_sparse_collection")

schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=False, max_length=100)
schema.add_field(field_name="embeddings", datatype=DataType.SPARSE_FLOAT_VECTOR)

index_params = client.prepare_index_params()

index_params.add_index(field_name="embeddings",
                               index_name="sparse_inverted_index",
                               index_type="SPARSE_INVERTED_INDEX",
                               metric_type="IP",
                               params={"drop_ratio_build": 0.2})
client.create_collection(
    collection_name="my_sparse_collection",
    schema=schema,
    index_params=index_params
)


In [3]:
embeddings_model = model.sparse.SpladeEmbeddingFunction(
    model_name="ibm-granite/granite-embedding-30m-sparse",
    device="cuda",
    batch_size=2,
    k_tokens_query=50,
    k_tokens_document=192
)

In [4]:
import os
os.getenv('CUDA_VISIBLE_DEVICES')

'1'

### Prepare documents to be ingested

In [5]:
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]
vecs = embeddings_model.encode_documents(docs)

In [6]:
for v in vecs:
    print(v)

<Compressed Sparse Row sparse array of dtype 'float32'
	with 192 stored elements and shape (50265,)>
  Coords	Values
  (0,)	0.0
  (1,)	0.0
  (2,)	0.0
  (3,)	0.0
  (4,)	0.0
  (5,)	0.0
  (6,)	0.0
  (7,)	0.0
  (8,)	0.0
  (9,)	0.0
  (10,)	0.0
  (11,)	0.0
  (12,)	0.0
  (13,)	0.0
  (14,)	0.0
  (15,)	0.0
  (16,)	0.0
  (17,)	0.0
  (18,)	0.0
  (19,)	0.0
  (20,)	0.0
  (21,)	0.0
  (22,)	0.0
  (23,)	0.0
  (24,)	0.0
  :	:
  (6441,)	0.6892693638801575
  (7147,)	0.5508409738540649
  (7350,)	1.2501308917999268
  (8408,)	0.27251625061035156
  (8882,)	1.219290018081665
  (9916,)	0.5613634586334229
  (10561,)	0.5431846976280212
  (11730,)	0.0014892152976244688
  (14578,)	0.020381949841976166
  (14619,)	0.3156839907169342
  (15221,)	0.03903009369969368
  (17194,)	0.4083267152309418
  (20257,)	0.21323853731155396
  (23931,)	0.002258133376017213
  (24649,)	1.0350993871688843
  (26101,)	0.9785782694816589
  (28034,)	0.18753792345523834
  (31024,)	0.021009808406233788
  (34580,)	0.22217050194740295
  (37283,)

In [20]:

doc_vector = [{"embeddings": doc_emb.reshape(1,50265), "id": f"item_{i}"} for i, doc_emb in enumerate(vecs)]

In [19]:
doc_vector[0]['embeddings'].reshape(1,50265)

<COOrdinate sparse array of dtype 'float32'
	with 192 stored elements and shape (1, 50265)>

In [21]:

client.insert(
    collection_name="my_sparse_collection",
    data=doc_vector
)


{'insert_count': 3, 'ids': ['456313106099863552', '456313106099863553', '456313106099863554'], 'cost': 0}

### Prepare search parameters

In [24]:
search_params = {
    "params": {"drop_ratio_search": 0.2},  # Additional optional search parameters
}


### Prepare the query vector

In [25]:

queries = [
      "When was artificial intelligence founded",
      "Where was Turing born?"
]
query_vector = embeddings_model.encode_documents(queries)

res = client.search(
    collection_name="my_sparse_collection",
    data=query_vector,
    limit=1, #top k documents to return
    output_fields=["id"],
    search_params=search_params,
)

for r in res:
    print(r)

[{'id': '456313106099863552', 'distance': 12.3641357421875, 'entity': {'id': 'item_0'}}]
[{'id': '456313106099863554', 'distance': 17.1358642578125, 'entity': {'id': 'item_2'}}]
